# Plantilla para la Tarea online BDA03

# Nombre del alumno:

Realiza las tareas que se plantean en cada ejercicio. En algunas tareas deberás completar las celdas que están incompletas en otras añadir nuevas celdas. Se trata de que implementes una serie de consultas con HQL (Hive) y Pig Latin.

Vamos a seguir utilizando el `dataset` de retrasos en vuelos en EEUU de la guía práctica. A modo de recordatorio, en el siguiente apartado, repetimos la explicación del significado de los campos.

# Dataset de retrasos en vuelos

Vamos a usar [este](https://www.kaggle.com/datasets/tylerx/flights-and-airports-data) de Kaggle
para aprender a usar tanto Hive como Pig. Kaggle es un sitio muy popular en ciencia de datos. En este sitio los científicos de datos pueden publicar y compartir sus trabajos. Además también se pueden proponer concursos en los que los participantes compiten en la construcción del mejor modelo para el problema propuesto.

El `dataset` contiene información sobre retrasos en vuelos en EEUU. Hay dos ficheros de interés: `airports.csv` y `flights.csv`.

El primero tiene información sobre los aeropuertos y consta de los siguientes campos:
   * airport_id: identificador del aeropuerto. Numérico, aunque se utilizará un campo `string` en Hive.
   * city: ciudad del aeropuerto.
   * state: estado del aeropuerto.
   * name: nombre del aeropuerto.
   
El fichero `flights` tiene la siguiente estructura:
   * DayofMonth: día del mes del vuelo.
   * DayOfWeek: día de la semana del vuelo.
   * Carrier: Identificador de la compañía aérea.
   * OriginAirportID: Identificador del aeropuerto de origen.
   * DestAirportID: Identificador del aeropuerto de destino.
   * DepDelay: Minutos de retraso en la salida de un vuelo (puede ser negativo si el vuelo sale antes de lo previsto).
   * ArrDelay: Minutos de retraso en la llegada de un vuelo (puede ser negativo si el vuelo sale antes de lo previsto).

El directorio `notebooks` contiene el `archiv.zip` con los dos ficheros. Para descargarlo de Kaggle hay que estar registrado y se ha incluido para que no tengas que registrarte.

## 1.- Realiza el proceso de preparación que se hizo en la guia práctica:

* Crea las celdas y muestra el resultado de su ejecución de la extracción de los ficheros del `dataset` de vuelos.
* Crea la base de datos de Hive y las tablas `airports` y `flights`. Presta atención a cambiar los comentarios y no simplemente copiar los de la guía.
* Carga las tablas y crea consultas de HQL que muestren 10 aeropuertos y 10 vuelos como se hizo en la guía práctica.
* Crea un `script` en Pig Latin que muestre 10 aeropuertos y 10 vuelos como se hizo en la guía práctica.

In [1]:
! apt update
! apt install unzip
! unzip -j -o archive.zip  airports.csv flights.csv

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:4 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2350 kB]33m
Get:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:6 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1273 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [2820 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [972 kB]m
Fetched 7750 kB in 7s (1089 kB/s)                                              m
Reading package lists... Done
Building dependency tree       
Reading state information... Done
181 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest versio

In [2]:
! wc -l airports.csv && head airports.csv 
! wc -l flights.csv && head flights.csv

366 airports.csv
airport_id,city,state,name
10165,Adak Island,AK,Adak
10299,Anchorage,AK,Ted Stevens Anchorage International
10304,Aniak,AK,Aniak Airport
10754,Barrow,AK,Wiley Post/Will Rogers Memorial
10551,Bethel,AK,Bethel Airport
10926,Cordova,AK,Merle K Mudhole Smith
14709,Deadhorse,AK,Deadhorse Airport
11336,Dillingham,AK,Dillingham Airport
11630,Fairbanks,AK,Fairbanks International
2702219 flights.csv
DayofMonth,DayOfWeek,Carrier,OriginAirportID,DestAirportID,DepDelay,ArrDelay
19,5,DL,11433,13303,-3,1
19,5,DL,14869,12478,0,-8
19,5,DL,14057,14869,-4,-15
19,5,DL,15016,11433,28,24
19,5,DL,11193,12892,-6,-11
19,5,DL,10397,15016,-1,-19
19,5,DL,15016,10397,0,-1
19,5,DL,10397,14869,15,24
19,5,DL,10397,10423,33,34


In [3]:
! hdfs dfs -mkdir -p /user/root/flights
! hdfs dfs -put -f ./airports.csv /user/root/flights/
! hdfs dfs -put -f ./flights.csv /user/root/flights/
! hdfs dfs -ls /user/root/flights/
! hdfs dfs -chmod 777 /user/root/flights

Found 2 items
-rw-r--r--   3 root supergroup      16308 2022-12-06 00:38 /user/root/flights/airports.csv
-rw-r--r--   3 root supergroup   72088113 2022-12-06 00:39 /user/root/flights/flights.csv


In [4]:
! beeline -u "jdbc:hive2://localhost:10000/" -e "\
CREATE DATABASE IF NOT EXISTS bda03 \
COMMENT 'Base de datos de la unidad BDA03' \
WITH DBPROPERTIES ('Creada por' = 'Javier Pérez', 'Fecha' = '20/12/22');"

Connecting to jdbc:hive2://localhost:10000/
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20221206003919_8cc19bde-d2eb-4c7f-aead-a8b9ca01e590): CREATE DATABASE IF NOT EXISTS bda03  COMMENT 'Base de datos de la unidad BDA03'  WITH DBPROPERTIES ('Creada por' = 'Javier P?rez', 'Fecha' = '20/12/22')
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20221206003919_8cc19bde-d2eb-4c7f-aead-a8b9ca01e590); Time taken: 0.034 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20221206003919_8cc19bde-d2eb-4c7f-aead-a8b9ca01e590): CREATE DATABASE IF NOT EXISTS bda03  COMMENT 'Base de datos de la unidad BDA03'  WITH DBPROPERTIES ('

In [5]:
! beeline -u "jdbc:hive2://localhost:10000/bda03" -e "\
DROP TABLE IF EXISTS airports; \
CREATE EXTERNAL TABLE IF NOT EXISTS airports (airportid STRING, city STRING, state STRING, airportname STRING) \
COMMENT 'USA Airports' \
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\,' \
TBLPROPERTIES ('Autor' = 'Javier Pérez', 'Fecha' = '20/12/2022', 'skip.header.line.count'='1');"

Connecting to jdbc:hive2://localhost:10000/bda03
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20221206004007_073490bc-49b5-4439-a2c4-dc1d7f41bae9): DROP TABLE IF EXISTS airports
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20221206004007_073490bc-49b5-4439-a2c4-dc1d7f41bae9); Time taken: 0.069 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20221206004007_073490bc-49b5-4439-a2c4-dc1d7f41bae9): DROP TABLE IF EXISTS airports
INFO  : Starting task [Stage-0:DDL] in serial mode
INFO  : Completed executing command(queryId=root_20221206004007_073490bc-49b5-4439-a2c4-dc1d7f41bae9); Time taken: 0.121 seconds
INFO  : O

In [6]:
! beeline -u "jdbc:hive2://localhost:10000/bda03" -e "\
DROP TABLE IF EXISTS flights; \
CREATE EXTERNAL TABLE IF NOT EXISTS flights (dayofmonth TINYINT, dayofweek TINYINT, carrier STRING, \
    depairportid STRING, arrairportid STRING, depdelay SMALLINT, arrdelay SMALLINT) \
COMMENT 'Flights' \
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\,' \
TBLPROPERTIES ('Autor' = 'Javier Pérez', 'Fecha' = '20/12/2022', 'skip.header.line.count'='1');"

Connecting to jdbc:hive2://localhost:10000/bda03
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20221206004020_a7427341-d982-473b-b68a-b4e5b01505a9): DROP TABLE IF EXISTS flights
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20221206004020_a7427341-d982-473b-b68a-b4e5b01505a9); Time taken: 0.05 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20221206004020_a7427341-d982-473b-b68a-b4e5b01505a9): DROP TABLE IF EXISTS flights
INFO  : Starting task [Stage-0:DDL] in serial mode
INFO  : Completed executing command(queryId=root_20221206004020_a7427341-d982-473b-b68a-b4e5b01505a9); Time taken: 0.061 seconds
INFO  : OK
I

In [7]:
! beeline -u "jdbc:hive2://localhost:10000/bda03" -e "\
LOAD DATA INPATH '/user/root/flights/airports.csv' OVERWRITE INTO TABLE airports;"

! beeline -u "jdbc:hive2://localhost:10000/bda03" -e "\
LOAD DATA INPATH '/user/root/flights/flights.csv' OVERWRITE INTO TABLE flights;"

Connecting to jdbc:hive2://localhost:10000/bda03
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20221206004059_88e29b55-d80b-4533-a749-83ef4b745b51): LOAD DATA INPATH '/user/root/flights/airports.csv' OVERWRITE INTO TABLE airports
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20221206004059_88e29b55-d80b-4533-a749-83ef4b745b51); Time taken: 0.046 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20221206004059_88e29b55-d80b-4533-a749-83ef4b745b51): LOAD DATA INPATH '/user/root/flights/airports.csv' OVERWRITE INTO TABLE airports
INFO  : Starting task [Stage-0:MOVE] in serial mode
INFO  : Loading data to table bda03

In [8]:
! beeline -u "jdbc:hive2://localhost:10000/bda03" -e "\
SELECT * FROM airports LIMIT 10"

! beeline -u "jdbc:hive2://localhost:10000/bda03" -e "\
SELECT * FROM flights LIMIT 10"

Connecting to jdbc:hive2://localhost:10000/bda03
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20221206004125_27b07d84-1d00-4945-bef0-48a6508d666d): SELECT * FROM airports LIMIT 10
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:airports.airportid, type:string, comment:null), FieldSchema(name:airports.city, type:string, comment:null), FieldSchema(name:airports.state, type:string, comment:null), FieldSchema(name:airports.airportname, type:string, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20221206004125_27b07d84-1d00-4945-bef0-48a6508d666d); Time taken: 0.116 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20221206004125_27b07d8

In [9]:
%%writefile flights.pig

-- resgistramos la librería PiggyBank para poder usar la función de carga CSVExcelStorage.
REGISTER piggybank.jar

/*
Leemos el fichero de airports.csv.

Usamos el loader CSVExcelStorage indicando el delimitador (,) y que se debe excluir la cabecera.
*/

AIRPORTS = LOAD '$airports_file' USING
       org.apache.pig.piggybank.storage.CSVExcelStorage(',', 'NO_MULTILINE', 'UNIX', 'SKIP_INPUT_HEADER')
       AS (airportid:chararray, city:chararray, state:chararray, airportname:chararray);

-- Leemos el fichero fligths.csv

FLIGHTS = LOAD '$flights_file' USING
       org.apache.pig.piggybank.storage.CSVExcelStorage(',', 'NO_MULTILINE', 'UNIX', 'SKIP_INPUT_HEADER')
       AS (dayofmonth:int, dayofweek:int, carrier:chararray, 
               depairportid:chararray, arrairportid:chararray, depdelay:int, arrdelay:int);


-- Probamos que podemos recuperar datos.
      
-- Nos quedamos con 10 aeropuertos
AIRPORTS_10 = LIMIT AIRPORTS 10;

-- Mostramos 10 aeropuertos
DUMP AIRPORTS_10;

-- Hacemos lo mismo con los vuelos
FLIGHTS_10 = LIMIT FLIGHTS 10;
DUMP FLIGHTS_10;

Overwriting flights.pig


In [10]:
! pig -x local -f flights.pig -param airports_file='airports.csv' -param flights_file='flights.csv' -param output_dir='pig/output/flights'

2022-12-06 00:42:25,165 INFO pig.ExecTypeProvider: Trying ExecType : LOCAL
2022-12-06 00:42:25,166 INFO pig.ExecTypeProvider: Picked LOCAL as the ExecType
2022-12-06 00:42:25,246 [main] INFO  org.apache.pig.Main - Apache Pig version 0.17.0 (r1797386) compiled Jun 02 2017, 15:41:58
2022-12-06 00:42:25,246 [main] INFO  org.apache.pig.Main - Logging error messages to: /media/notebooks/pig_1670283745243.log
2022-12-06 00:42:25,263 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - user.name is deprecated. Instead, use mapreduce.job.user.name
2022-12-06 00:42:25,455 [main] INFO  org.apache.pig.impl.util.Utils - Default bootup file /root/.pigbootup not found
2022-12-06 00:42:25,580 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2022-12-06 00:42:25,582 [main] INFO  org.apache.pig.backend.hadoop.executionengine.HExecutionEngine - Connecting to hadoop file system at: file:///
2022-12-06 00:

## 2.- Con una consulta de HQL muestra: La cinco compañías que más vuelos retrasados tienen.

* El campo `carrier` contiene la compañía aérea.
* Vamos a considerar que un vuelo llega con retraso cuando el vuelo llega más de 15 minutos tarde (campo `arrdelay` > 15).

Se espera el siguiente resultado:

![solución 2](./img/2.png)

In [12]:
! beeline -u "jdbc:hive2://localhost:10000/bda03" -e "\
SELECT carrier, COUNT(*) AS delayed_flights \
FROM flights WHERE arrdelay > 15 \
GROUP BY carrier \
ORDER BY delayed_flights DESC LIMIT 5;"

Connecting to jdbc:hive2://localhost:10000/bda03
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20221206005528_bdafec5c-d458-47be-a233-ca229a415e31): SELECT carrier, COUNT(*) AS delayed_flights  FROM flights WHERE arrdelay > 15  GROUP BY carrier  ORDER BY delayed_flights DESC LIMIT 5
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:carrier, type:string, comment:null), FieldSchema(name:delayed_flights, type:bigint, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20221206005528_bdafec5c-d458-47be-a233-ca229a415e31); Time taken: 0.14 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20221206005528_bdafec5c-d458-47be-a233-ca229a415e31): SEL

## 3.- Con una consulta de HQL muestra: Las 5 compañías que mejor recuperación de tiempo en vuelo tienen.

* Se considera que se ha recuperado el tiempo de un vuelo cuando habiendo salido con retraso (`depdelay` > 15), llega sin retraso (`arraydelay` <= 15).
* Se trata de que muestres las 5 compañías que han recuperado el tiempo en un mayor porcentaje de vuelos que salieron retrasados.

El resultado esperado es el siguiente:

![solución 3](./img/3.png)

In [14]:
 ! beeline -u "jdbc:hive2://localhost:10000/bda03" -e "\
SELECT carrier, SUM(IF(arrdelay <=15, 1, 0)) / count(*) AS percent_recovered \
FROM flights WHERE depdelay > 15 \
GROUP BY carrier \
ORDER BY percent_recovered DESC LIMIT 5;"

Connecting to jdbc:hive2://localhost:10000/bda03
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20221206010649_c1b3a513-0bed-4381-bcb3-ba3a0055ac2c): SELECT carrier, SUM(IF(arrdelay <=15, 1, 0)) / count(*) AS percent_recovered  FROM flights WHERE depdelay > 15  GROUP BY carrier  ORDER BY percent_recovered DESC LIMIT 5
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:carrier, type:string, comment:null), FieldSchema(name:percent_recovered, type:double, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20221206010649_c1b3a513-0bed-4381-bcb3-ba3a0055ac2c); Time taken: 0.138 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20221206010649_c1b3

## 4.- Resuelve el ejercicio 2 con Pig Latin

El resultado esperado es:

![solución 4](./img/4.png)

In [21]:
%%writefile flights.pig

-- resgistramos la librería PiggyBank para poder usar la función de carga CSVExcelStorage.
REGISTER piggybank.jar

-- Leemos el fichero fligths.csv

FLIGHTS = LOAD '$flights_file' USING
       org.apache.pig.piggybank.storage.CSVExcelStorage(',', 'NO_MULTILINE', 'UNIX', 'SKIP_INPUT_HEADER')
       AS (dayofmonth:int, dayofweek:int, carrier:chararray, 
               depairportid:chararray, arrairportid:chararray, depdelay:int, arrdelay:int);
        

DELAYED_FLIGHTS = FILTER FLIGHTS BY arrdelay > 15;

DELAYED_FLIGHTS_BY_CARRIER = GROUP DELAYED_FLIGHTS BY carrier;
DELAYED_FLIGHTS_BY_CARRIER = FOREACH DELAYED_FLIGHTS_BY_CARRIER GENERATE group AS carrier, COUNT(DELAYED_FLIGHTS) AS delayed_flights;
DELAYED_FLIGHTS_BY_CARRIER = ORDER DELAYED_FLIGHTS_BY_CARRIER BY delayed_flights DESC;

TOP_DELAYED_FLIGHTS_BY_CARRIER = LIMIT DELAYED_FLIGHTS_BY_CARRIER 5;
DUMP TOP_DELAYED_FLIGHTS_BY_CARRIER;

Overwriting flights.pig


In [22]:
! pig -x local -f flights.pig -param flights_file='flights.csv' -param output_dir='pig/output/flights'

2022-12-06 01:21:08,367 INFO pig.ExecTypeProvider: Trying ExecType : LOCAL
2022-12-06 01:21:08,367 INFO pig.ExecTypeProvider: Picked LOCAL as the ExecType
2022-12-06 01:21:08,418 [main] INFO  org.apache.pig.Main - Apache Pig version 0.17.0 (r1797386) compiled Jun 02 2017, 15:41:58
2022-12-06 01:21:08,418 [main] INFO  org.apache.pig.Main - Logging error messages to: /media/notebooks/pig_1670286068415.log
2022-12-06 01:21:08,433 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - user.name is deprecated. Instead, use mapreduce.job.user.name
2022-12-06 01:21:08,554 [main] INFO  org.apache.pig.impl.util.Utils - Default bootup file /root/.pigbootup not found
2022-12-06 01:21:08,595 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2022-12-06 01:21:08,597 [main] INFO  org.apache.pig.backend.hadoop.executionengine.HExecutionEngine - Connecting to hadoop file system at: file:///
2022-12-06 01:

2022-12-06 01:21:09,984 [Thread-6] INFO  org.apache.hadoop.mapred.LocalJobRunner - Waiting for map tasks
2022-12-06 01:21:09,985 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.LocalJobRunner - Starting task: attempt_local248586027_0001_m_000000_0
2022-12-06 01:21:10,014 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter - File Output Committer Algorithm version is 2
2022-12-06 01:21:10,014 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter - FileOutputCommitter skip cleanup _temporary folders under output directory:false, ignore cleanup failures: false
2022-12-06 01:21:10,025 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.Task -  Using ResourceCalculatorProcessTree : [ ]
2022-12-06 01:21:10,032 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.MapTask - Processing split: Number of splits :1
Total Length = 33554432
Input s

2022-12-06 01:21:14,225 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - 12% complete
2022-12-06 01:21:14,228 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - Running jobs are [job_local248586027_0001]
2022-12-06 01:21:17,495 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.LocalJobRunner - 
2022-12-06 01:21:17,495 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.MapTask - Starting flush of map output
2022-12-06 01:21:17,495 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.MapTask - Spilling map output
2022-12-06 01:21:17,495 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.MapTask - bufstart = 0; bufend = 2384890; bufvoid = 104857600
2022-12-06 01:21:17,495 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.MapTask - kvstart = 26214396(104857584); kvend = 25260444(101041776); length = 953953/6553600


2022-12-06 01:21:18,251 [localfetcher#1] INFO  org.apache.hadoop.mapreduce.task.reduce.LocalFetcher - localfetcher#1 about to shuffle output of map attempt_local248586027_0001_m_000000_0 decomp: 230 len: 234 to MEMORY
2022-12-06 01:21:18,255 [localfetcher#1] INFO  org.apache.hadoop.mapreduce.task.reduce.InMemoryMapOutput - Read 230 bytes from map-output for attempt_local248586027_0001_m_000000_0
2022-12-06 01:21:18,256 [localfetcher#1] INFO  org.apache.hadoop.mapreduce.task.reduce.MergeManagerImpl - closeInMemoryFile -> map-output of size: 230, inMemoryMapOutputs.size() -> 1, commitMemory -> 0, usedMemory ->230
2022-12-06 01:21:18,259 [localfetcher#1] INFO  org.apache.hadoop.mapreduce.task.reduce.LocalFetcher - localfetcher#1 about to shuffle output of map attempt_local248586027_0001_m_000002_0 decomp: 170 len: 174 to MEMORY
2022-12-06 01:21:18,260 [localfetcher#1] INFO  org.apache.hadoop.mapreduce.task.reduce.InMemoryMapOutput - Read 170 bytes from map-output for attempt_local24858602

2022-12-06 01:21:18,503 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - 1 map-reduce job(s) waiting for submission.
2022-12-06 01:21:18,506 [JobControl] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTracker metrics system already initialized!
2022-12-06 01:21:18,512 [JobControl] WARN  org.apache.hadoop.mapreduce.JobResourceUploader - No job jar file set.  User classes may not be found. See Job or Job#setJar(String).
2022-12-06 01:21:18,516 [JobControl] INFO  org.apache.hadoop.mapreduce.lib.input.FileInputFormat - Total input files to process : 1
2022-12-06 01:21:18,516 [JobControl] INFO  org.apache.pig.backend.hadoop.executionengine.util.MapRedUtil - Total input paths to process : 1
2022-12-06 01:21:18,516 [JobControl] INFO  org.apache.pig.backend.hadoop.executionengine.util.MapRedUtil - Total input paths (combined) to process : 1
2022-12-06 01:21:18,518 [JobControl] INFO  org.apache.hadoop.mapreduce.JobSubmitter - number of 

2022-12-06 01:21:18,695 [pool-9-thread-1] INFO  org.apache.hadoop.mapred.Task - Task:attempt_local1085178813_0002_r_000000_0 is done. And is in the process of committing
2022-12-06 01:21:18,699 [pool-9-thread-1] INFO  org.apache.hadoop.mapred.LocalJobRunner - 1 / 1 copied.
2022-12-06 01:21:18,699 [pool-9-thread-1] INFO  org.apache.hadoop.mapred.Task - Task attempt_local1085178813_0002_r_000000_0 is allowed to commit now
2022-12-06 01:21:18,702 [pool-9-thread-1] INFO  org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter - Saved output of task 'attempt_local1085178813_0002_r_000000_0' to file:/tmp/temp1503319484/tmp1158266018
2022-12-06 01:21:18,702 [pool-9-thread-1] INFO  org.apache.hadoop.mapred.LocalJobRunner - reduce > reduce
2022-12-06 01:21:18,702 [pool-9-thread-1] INFO  org.apache.hadoop.mapred.Task - Task 'attempt_local1085178813_0002_r_000000_0' done.
2022-12-06 01:21:18,703 [pool-9-thread-1] INFO  org.apache.hadoop.mapred.Task - Final Counters for attempt_local1085178813_

2022-12-06 01:21:19,048 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.Task - Task:attempt_local732394886_0003_m_000000_0 is done. And is in the process of committing
2022-12-06 01:21:19,049 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.LocalJobRunner - map
2022-12-06 01:21:19,049 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.Task - Task 'attempt_local732394886_0003_m_000000_0' done.
2022-12-06 01:21:19,049 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.Task - Final Counters for attempt_local732394886_0003_m_000000_0: Counters: 18
	File System Counters
		FILE: Number of bytes read=72102594
		FILE: Number of bytes written=1863651
		FILE: Number of read operations=0
		FILE: Number of large read operations=0
		FILE: Number of write operations=0
	Map-Reduce Framework
		Map input records=16
		Map output records=16
		Map output bytes=288
		Map output materialized bytes=106
		Input split bytes=379
		Comb

2022-12-06 01:21:19,209 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - HadoopJobId: job_local732394886_0003
2022-12-06 01:21:19,209 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - Processing aliases DELAYED_FLIGHTS_BY_CARRIER
2022-12-06 01:21:19,209 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - detailed locations: M: DELAYED_FLIGHTS_BY_CARRIER[17,29] C:  R: 
2022-12-06 01:21:19,210 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - 75% complete
2022-12-06 01:21:19,212 [main] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTracker metrics system already initialized!
2022-12-06 01:21:19,213 [main] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTracker metrics system already initialized!
2022-12-06 01:21:19,214 [main] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTra

2022-12-06 01:21:19,354 [pool-15-thread-1] INFO  org.apache.hadoop.mapred.Task -  Using ResourceCalculatorProcessTree : [ ]
2022-12-06 01:21:19,354 [pool-15-thread-1] INFO  org.apache.hadoop.mapred.ReduceTask - Using ShuffleConsumerPlugin: org.apache.hadoop.mapreduce.task.reduce.Shuffle@23ffd70d
2022-12-06 01:21:19,354 [pool-15-thread-1] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTracker metrics system already initialized!
2022-12-06 01:21:19,355 [pool-15-thread-1] INFO  org.apache.hadoop.mapreduce.task.reduce.MergeManagerImpl - MergerManager: memoryLimit=652528832, maxSingleShuffleLimit=163132208, mergeThreshold=430669056, ioSortFactor=10, memToMemMergeOutputsThreshold=10
2022-12-06 01:21:19,355 [EventFetcher for fetching Map Completion Events] INFO  org.apache.hadoop.mapreduce.task.reduce.EventFetcher - attempt_local807153516_0004_r_000000_0 Thread started: EventFetcher for fetching Map Completion Events
2022-12-06 01:21:19,357 [localfetcher#4] INFO  org.apache.hado

2022-12-06 01:21:19,556 [main] WARN  org.apache.pig.data.SchemaTupleBackend - SchemaTupleBackend has already been initialized
2022-12-06 01:21:19,558 [main] INFO  org.apache.hadoop.mapreduce.lib.input.FileInputFormat - Total input files to process : 1
2022-12-06 01:21:19,558 [main] INFO  org.apache.pig.backend.hadoop.executionengine.util.MapRedUtil - Total input paths to process : 1
(WN,127601)
(AA,59842)
(DL,57668)
(UA,57367)
(US,40943)
2022-12-06 01:21:19,591 [main] INFO  org.apache.pig.Main - Pig script completed in 11 seconds and 361 milliseconds (11361 ms)


## 5.- Resuelve el ejercicio 3 con Pig Latin

Se espera el siguiente resultado:

![solución 5](./img/5.png)

In [106]:
%%writefile flights.pig

-- resgistramos la librería PiggyBank para poder usar la función de carga CSVExcelStorage.
REGISTER piggybank.jar

-- Leemos el fichero fligths.csv

FLIGHTS = LOAD '$flights_file' USING
       org.apache.pig.piggybank.storage.CSVExcelStorage(',', 'NO_MULTILINE', 'UNIX', 'SKIP_INPUT_HEADER')
       AS (dayofmonth:int, dayofweek:int, carrier:chararray, 
               depairportid:chararray, arrairportid:chararray, depdelay:int, arrdelay:int);
        

DELAYED_DEP_FLIGHTS = FILTER FLIGHTS BY depdelay > 15;
ONTIME_ARR_FLIGHTS = FILTER DELAYED_DEP_FLIGHTS BY arrdelay <= 15;


DELAYED_DEP_FLIGHTS_BY_CARRIER = GROUP DELAYED_DEP_FLIGHTS BY carrier;

DELAYED_DEP_FLIGHTS_BY_CARRIER = FOREACH DELAYED_DEP_FLIGHTS_BY_CARRIER {
    carrier = group;
    ontime_flights = FOREACH DELAYED_DEP_FLIGHTS.arrdelay GENERATE ($0 <= 15 ? 1 : 0);
    ontime_flights = SUM(ontime_flights);
    delayed_flights = COUNT(DELAYED_DEP_FLIGHTS);
    
    GENERATE group AS carrier, (float)ontime_flights / delayed_flights AS percent_recovered;
};

DELAYED_DEP_FLIGHTS_BY_CARRIER = ORDER DELAYED_DEP_FLIGHTS_BY_CARRIER BY percent_recovered DESC;

TOP_DELAYED_DEP_FLIGHTS_BY_CARRIER = LIMIT DELAYED_DEP_FLIGHTS_BY_CARRIER 5;
DUMP TOP_DELAYED_DEP_FLIGHTS_BY_CARRIER;

Overwriting flights.pig


In [107]:
! pig -x local -f flights.pig -param flights_file='flights.csv' -param output_dir='pig/output/flights'

2022-12-06 02:09:54,593 INFO pig.ExecTypeProvider: Trying ExecType : LOCAL
2022-12-06 02:09:54,594 INFO pig.ExecTypeProvider: Picked LOCAL as the ExecType
2022-12-06 02:09:54,683 [main] INFO  org.apache.pig.Main - Apache Pig version 0.17.0 (r1797386) compiled Jun 02 2017, 15:41:58
2022-12-06 02:09:54,683 [main] INFO  org.apache.pig.Main - Logging error messages to: /media/notebooks/pig_1670288994679.log
2022-12-06 02:09:54,704 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - user.name is deprecated. Instead, use mapreduce.job.user.name
2022-12-06 02:09:54,951 [main] INFO  org.apache.pig.impl.util.Utils - Default bootup file /root/.pigbootup not found
2022-12-06 02:09:55,040 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2022-12-06 02:09:55,042 [main] INFO  org.apache.pig.backend.hadoop.executionengine.HExecutionEngine - Connecting to hadoop file system at: file:///
2022-12-06 02:

2022-12-06 02:09:56,505 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter - File Output Committer Algorithm version is 2
2022-12-06 02:09:56,505 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter - FileOutputCommitter skip cleanup _temporary folders under output directory:false, ignore cleanup failures: false
2022-12-06 02:09:56,521 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.Task -  Using ResourceCalculatorProcessTree : [ ]
2022-12-06 02:09:56,528 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.MapTask - Processing split: Number of splits :1
Total Length = 33554432
Input split[0]:
   Length = 33554432
   ClassName: org.apache.hadoop.mapreduce.lib.input.FileSplit
   Locations:

-----------------------

2022-12-06 02:09:56,542 [LocalJobRunner Map Task Executor #0] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.PigRecor

2022-12-06 02:10:00,598 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - 12% complete
2022-12-06 02:10:00,599 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - Running jobs are [job_local11709535_0001]
2022-12-06 02:10:03,311 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.LocalJobRunner - 
2022-12-06 02:10:03,311 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.MapTask - Starting flush of map output
2022-12-06 02:10:03,311 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.MapTask - Spilling map output
2022-12-06 02:10:03,311 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.MapTask - bufstart = 0; bufend = 7544062; bufvoid = 104857600
2022-12-06 02:10:03,311 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.MapTask - kvstart = 26214396(104857584); kvend = 25270236(101080944); length = 944161/6553600
2

2022-12-06 02:10:04,006 [localfetcher#1] INFO  org.apache.hadoop.mapreduce.task.reduce.LocalFetcher - localfetcher#1 about to shuffle output of map attempt_local11709535_0001_m_000001_0 decomp: 8016146 len: 8016150 to MEMORY
2022-12-06 02:10:04,014 [localfetcher#1] INFO  org.apache.hadoop.mapreduce.task.reduce.InMemoryMapOutput - Read 8016146 bytes from map-output for attempt_local11709535_0001_m_000001_0
2022-12-06 02:10:04,015 [localfetcher#1] INFO  org.apache.hadoop.mapreduce.task.reduce.MergeManagerImpl - closeInMemoryFile -> map-output of size: 8016146, inMemoryMapOutputs.size() -> 1, commitMemory -> 0, usedMemory ->8016146
2022-12-06 02:10:04,021 [localfetcher#1] INFO  org.apache.hadoop.mapreduce.task.reduce.LocalFetcher - localfetcher#1 about to shuffle output of map attempt_local11709535_0001_m_000000_0 decomp: 8983885 len: 8983889 to MEMORY
2022-12-06 02:10:04,027 [localfetcher#1] INFO  org.apache.hadoop.mapreduce.task.reduce.InMemoryMapOutput - Read 8983885 bytes from map-out

2022-12-06 02:10:06,087 [JobControl] INFO  org.apache.hadoop.mapreduce.JobSubmitter - number of splits:1
2022-12-06 02:10:06,106 [JobControl] INFO  org.apache.hadoop.mapreduce.JobSubmitter - Submitting tokens for job: job_local348278526_0002
2022-12-06 02:10:06,106 [JobControl] INFO  org.apache.hadoop.mapreduce.JobSubmitter - Executing with tokens: []
2022-12-06 02:10:06,195 [JobControl] INFO  org.apache.hadoop.mapreduce.Job - The url to track the job: http://localhost:8080/
2022-12-06 02:10:06,196 [Thread-17] INFO  org.apache.hadoop.mapred.LocalJobRunner - OutputCommitter set in config null
2022-12-06 02:10:06,201 [Thread-17] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
2022-12-06 02:10:06,202 [Thread-17] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2022-12-06 02:10:06,202 [Thread-17] INFO  org.apache.hadoop.conf.Conf

2022-12-06 02:10:06,298 [pool-9-thread-1] INFO  org.apache.hadoop.mapred.Task - Task:attempt_local348278526_0002_r_000000_0 is done. And is in the process of committing
2022-12-06 02:10:06,302 [pool-9-thread-1] INFO  org.apache.hadoop.mapred.LocalJobRunner - 1 / 1 copied.
2022-12-06 02:10:06,302 [pool-9-thread-1] INFO  org.apache.hadoop.mapred.Task - Task attempt_local348278526_0002_r_000000_0 is allowed to commit now
2022-12-06 02:10:06,307 [pool-9-thread-1] INFO  org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter - Saved output of task 'attempt_local348278526_0002_r_000000_0' to file:/tmp/temp-961947915/tmp-1585251709
2022-12-06 02:10:06,321 [pool-9-thread-1] INFO  org.apache.hadoop.mapred.LocalJobRunner - reduce > reduce
2022-12-06 02:10:06,321 [pool-9-thread-1] INFO  org.apache.hadoop.mapred.Task - Task 'attempt_local348278526_0002_r_000000_0' done.
2022-12-06 02:10:06,321 [pool-9-thread-1] INFO  org.apache.hadoop.mapred.Task - Final Counters for attempt_local348278526_0002

2022-12-06 02:10:06,645 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.Task - Final Counters for attempt_local1894161632_0003_m_000000_0: Counters: 18
	File System Counters
		FILE: Number of bytes read=107682160
		FILE: Number of bytes written=37434840
		FILE: Number of read operations=0
		FILE: Number of large read operations=0
		FILE: Number of write operations=0
	Map-Reduce Framework
		Map input records=16
		Map output records=16
		Map output bytes=224
		Map output materialized bytes=86
		Input split bytes=378
		Combine input records=16
		Combine output records=5
		Spilled Records=5
		Failed Shuffles=0
		Merged Map outputs=0
		GC time elapsed (ms)=0
		Total committed heap usage (bytes)=820510720
	File Input Format Counters 
		Bytes Read=0
2022-12-06 02:10:06,645 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.LocalJobRunner - Finishing task: attempt_local1894161632_0003_m_000000_0
2022-12-06 02:10:06,646 [Thread-24] INFO  org.apache.hadoop.

2022-12-06 02:10:06,800 [main] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTracker metrics system already initialized!
2022-12-06 02:10:06,801 [main] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTracker metrics system already initialized!
2022-12-06 02:10:06,805 [main] INFO  org.apache.pig.tools.pigstats.mapreduce.MRScriptState - Pig script settings are added to the job
2022-12-06 02:10:06,805 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.JobControlCompiler - mapred.job.reduce.markreset.buffer.percent is not set, set to default 0.3
2022-12-06 02:10:06,805 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.JobControlCompiler - Reduce phase detected, estimating # of required reducers.
2022-12-06 02:10:06,805 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.JobControlCompiler - Setting Parallelism to 1
2022-12-06 02:10:06,806 [main] INFO  org.apache.pig.backend.hadoop.executionengi

2022-12-06 02:10:07,084 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - HadoopJobId: job_local1097056699_0004
2022-12-06 02:10:07,084 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - Processing aliases DELAYED_DEP_FLIGHTS_BY_CARRIER
2022-12-06 02:10:07,084 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - detailed locations: M: DELAYED_DEP_FLIGHTS_BY_CARRIER[28,33] C:  R: 
2022-12-06 02:10:07,086 [main] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTracker metrics system already initialized!
2022-12-06 02:10:07,087 [main] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTracker metrics system already initialized!
2022-12-06 02:10:07,088 [main] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTracker metrics system already initialized!
2022-12-06 02:10:07,093 [main] INFO  org.apache.pig.backend.hadoop.executionengin

In [29]:
! pig -x local -f flights.pig -param flights_file='flights.csv' -param output_dir='pig/output/flights'

2022-12-06 01:41:40,033 INFO pig.ExecTypeProvider: Trying ExecType : LOCAL
2022-12-06 01:41:40,034 INFO pig.ExecTypeProvider: Picked LOCAL as the ExecType
2022-12-06 01:41:40,118 [main] INFO  org.apache.pig.Main - Apache Pig version 0.17.0 (r1797386) compiled Jun 02 2017, 15:41:58
2022-12-06 01:41:40,118 [main] INFO  org.apache.pig.Main - Logging error messages to: /media/notebooks/pig_1670287300114.log
2022-12-06 01:41:40,138 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - user.name is deprecated. Instead, use mapreduce.job.user.name
2022-12-06 01:41:40,396 [main] INFO  org.apache.pig.impl.util.Utils - Default bootup file /root/.pigbootup not found
2022-12-06 01:41:40,464 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2022-12-06 01:41:40,466 [main] INFO  org.apache.pig.backend.hadoop.executionengine.HExecutionEngine - Connecting to hadoop file system at: file:///
2022-12-06 01: